In [5]:
import torch
# import numpy as np
# import matplotlib.pyplot as plt
# import spectral
# from scipy.io import loadmat
# import os
# from HSI_class import HSI

# Check if GPU is available
print("GPU Available:", torch.cuda.is_available())

# If available, print the GPU name
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

GPU Available: False
